In [1]:
#Import packages
import sys
import numpy as np
import pandas as pd
import string 
##loading stopwords 
stopwords = pd.read_csv('stop_words.csv')['word'].tolist()

In [2]:
with open('aljazeera-khashoggi.txt',encoding = 'utf-8',errors='ignore') as f:##read in the data 
    aljazeera = f.read()
with open('bbc-khashoggi.txt',encoding = 'utf-8',errors='ignore') as f:
    bbc = f.read()
with open('breitbart-khashoggi.txt',encoding = 'utf-8',errors='ignore') as f:
    breitbart = f.read()
with open('cnn-khashoggi.txt',encoding = 'utf-8',errors='ignore') as f:
    cnn = f.read()
with open('fox-khashoggi.txt',encoding = 'utf-8',errors='ignore') as f:
    fox = f.read()

In [18]:
def tokenize(text=None,choice=0):## tokenize function splits the string passed as argument after performing some modifications - changing all to lower case,
    text = text.lower()##string to lower case
    text = text.replace('.','')##Changing certaing punctuations
    text = text.replace('\"','')  
    text = text.replace('\'','')  
    text = text.replace('?','')  
    text = text.replace('0', '')
    text = text.replace('1', '')
    text = text.replace('2', '')
    text = text.replace('3', '')
    text = text.replace('4', '')
    text = text.replace('5', '')
    text = text.replace('6', '')
    text = text.replace('7', '')
    text = text.replace('8', '')
    text = text.replace('9', '')
    text = text.replace('$', '')
    text = text.replace('(','')
    text = text.replace(')','')
    text = text.replace('[','')
    text = text.replace(']','')
    text = text.replace('“','')
    text = text.replace('”','')
    text = text.replace(',', '')
    text_with = text.split()
    if(choice==0):##with stopwords
        list_final = text_with
    if(choice==1):##without stopwords
        text_without= [word for word in text_with if word not in stopwords]##Not including stop words in the modified string
        list_final =text_without
    return list_final#Returning the string

In [19]:
def convert_text_to_dtm(txt, choice=0):##convert_text_to_dtm will convert the string passed as an argument into a document-term matrix.
    d = dict()###Initialising empty dictionary
    for word in tokenize(txt,choice):
        if word in d:
            d[word][0] += 1
        else:
            d[word] = [1]
    return pd.DataFrame(d)

In [20]:
def gen_DTM(texts=None, choice=0):## gen_DTM will create a document term matrix for all the strings present in the argument.
    DTM = pd.DataFrame() #Initializing empty Dataframe
    for text in texts:
        entry = convert_text_to_dtm(text,choice)###Appending each result to the dataframe initialized earlier
        DTM = DTM.append(pd.DataFrame(entry),ignore_index=True,sort=True) # Row bind
    
    DTM.fillna(0, inplace=True) # Fill in any missing values with 0s (i.e. when a word is in one text but not another)
    return DTM

In [21]:
def cosine(a,b):
    '''
    Calculates cosine distance
    ----------------------------
    Args: 
    A set of string documents in this case
    ----------------------------
    Output:
    Cosine distance coefficients
    '''
    cos = np.dot(a,b)/(np.sqrt(np.dot(a,a)) * np.sqrt(np.dot(b,b)) )##Calculating the cos
    return cos

In [7]:
##with stopwords

In [23]:
A = gen_DTM([aljazeera, bbc, breitbart, cnn, fox],0)#Index the pandas dataframe to draw out a vector
a0 = A.iloc[0].values
b0 = A.iloc[1].values
c0 = A.iloc[2].values
d0 = A.iloc[3].values
e0 = A.iloc[4].values
df_nostop = {'Aljazeera':a0,'BBC':b0,'Breitbart':c0,'CNN':d0,'Fox':e0} ###Create a new dictionary to assign the corresponding name for each vector
df_nostop = pd.DataFrame(df_nostop)#Transfer the dictionary into dataframe form
df_nostop_corr = df_nostop.corr(method=cosine).round(3)#Make cosine correlation dataframe for the vectors
df_nostop_corr#Dataframe for the correlation betetween vectors(not includes stopwords)

,Aljazeera,BBC,Breitbart,CNN,Fox
Aljazeera,1.000,0.871,0.838,0.735,0.842
BBC,0.871,1.000,0.897,0.744,0.886
Breitbart,0.838,0.897,1.000,0.679,0.867
CNN,0.735,0.744,0.679,1.000,0.736
Fox,0.842,0.886,0.867,0.736,1.000


In [9]:
##without stopwords

In [24]:
B = gen_DTM([aljazeera, bbc, breitbart, cnn, fox],1)#Index the pandas dataframe to draw out a vector
a1 = B.iloc[0].values
b1= B.iloc[1].values
c1 = B.iloc[2].values
d1 = B.iloc[3].values
e1 = B.iloc[4].values
df_stop = {'Aljazeera':a1,'BBC':b1,'Breitbart':c1,'CNN':d1,'Fox':e1} ###Create a new dictionary to assign the corresponding name for each vector
df_stop = pd.DataFrame(df_stop)#Transfer the dictionary into dataframe form
df_stop_corr = df_stop.corr(method=cosine).round(3)#Make cosine correlation dataframe for the vectors
df_stop_corr#Dataframe for the correlation betetween vectors(not includes stopwords)

,Aljazeera,BBC,Breitbart,CNN,Fox
Aljazeera,1.000,0.678,0.587,0.532,0.678
BBC,0.678,1.000,0.581,0.502,0.625
Breitbart,0.587,0.581,1.000,0.368,0.546
CNN,0.532,0.502,0.368,1.000,0.516
Fox,0.678,0.625,0.546,0.516,1.000


Does each news site report on these stories in a similar way? Which news sites talk about the Khashoggi scandal in similar/dissimilar ways? If you change what words you remove, does the picture of similarity change?

Conclusion:
Each news site report on the story in different ways. There is a highest similarity between BBC and Aljazeera talking about the Khashoggi scandal. Among these news sites, the CNN and BBC have the lowest similarity talking about the scandal. Changing the words removed shall slightly change the specific parameter, yet the picture of similarity will not be changed.